In [4]:
print("2022-03-15 00:00:00"[0:7])

2022-03


In [ ]:

from langchain.embeddings import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
print(embedding.model)

text-embedding-ada-002


this is with 72 or 73 questions

In [4]:
data = {'generate_qa_comparison_party': {'cosine_RAG_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 12, 'No': 4}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 15, 'No': 1}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 10, 'No': 7}, 'relevance': {'Yes': 15, 'No': 2}}}, 'generate_qa_comparison_person': {'cosine_RAG_response': {'factuality': {'Yes': 17, 'No': 4}, 'relevance': {'Yes': 18, 'No': 3}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 16, 'No': 5}, 'relevance': {'Yes': 15, 'No': 6}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 13, 'No': 8}, 'relevance': {'Yes': 19, 'No': 2}}}, 'generate_qa_inference_person': {'cosine_RAG_response': {'factuality': {'Yes': 7, 'No': 7}, 'relevance': {'Yes': 8, 'No': 6}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 9, 'No': 5}, 'relevance': {'Yes': 9, 'No': 5}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 9, 'No': 5}, 'relevance': {'Yes': 8, 'No': 6}}}, 'generate_qa_inference_party': {'cosine_RAG_response': {'factuality': {'Yes': 3, 'No': 6}, 'relevance': {'Yes': 4, 'No': 5}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 4, 'No': 5}, 'relevance': {'Yes': 4, 'No': 5}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 3, 'No': 5}, 'relevance': {'Yes': 5, 'No': 3}}}, 'generate_qa_temporal': {'cosine_RAG_response': {'factuality': {'Yes': 1, 'No': 7}, 'relevance': {'Yes': 1, 'No': 7}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 5, 'No': 3}, 'relevance': {'Yes': 5, 'No': 3}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 4, 'No': 4}, 'relevance': {'Yes': 4, 'No': 4}}}}

# Output the data in the required format
for qa_type, response_data in data.items():
    print(f"qa_type: {qa_type}")
    for response_type, counts in response_data.items():
        factuality_yes = counts["factuality"]["Yes"]
        factuality_no = counts["factuality"]["No"]
        relevance_yes = counts["relevance"]["Yes"]
        relevance_no = counts["relevance"]["No"]
        
        # Output the counts in the desired format
        print(f"  {response_type.replace('_', ' ').title()} - Factuality: {factuality_yes}/{factuality_no}")
        print(f"  {response_type.replace('_', ' ').title()} - Relevance: {relevance_yes}/{relevance_no}")
    print()  # Line break for better readability

    print(data)

qa_type: generate_qa_comparison_party
  Cosine Rag Response - Factuality: 13/3
  Cosine Rag Response - Relevance: 12/4
  Graph Rag Bm25 Response - Factuality: 13/3
  Graph Rag Bm25 Response - Relevance: 15/1
  Graph Rag Cosine Response - Factuality: 10/7
  Graph Rag Cosine Response - Relevance: 15/2

{'generate_qa_comparison_party': {'cosine_RAG_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 12, 'No': 4}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 15, 'No': 1}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 10, 'No': 7}, 'relevance': {'Yes': 15, 'No': 2}}}, 'generate_qa_comparison_person': {'cosine_RAG_response': {'factuality': {'Yes': 17, 'No': 4}, 'relevance': {'Yes': 18, 'No': 3}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 16, 'No': 5}, 'relevance': {'Yes': 15, 'No': 6}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 13, 'No': 8}, 'relevance': {'Yes': 19, 'No': 2}}}, 'generate_qa_inference_person': 

In [3]:
def combine_qa_types(data):
    # Define which query types to combine
    combine_pairs = [
        ('generate_qa_comparison_party', 'generate_qa_comparison_person'),
        ('generate_qa_inference_party', 'generate_qa_inference_person'),
        # Add other pairs to combine as needed
    ]

    # Initialize a new dictionary for combined results
    combined_data = {}

    # Loop through the defined query type pairs
    for first_qa_type, second_qa_type in combine_pairs:
        # Ensure that both query types are in the data
        if first_qa_type in data and second_qa_type in data:
            combined_data[first_qa_type] = {}
            
            # Combine the data for each response type
            for response_type in data[first_qa_type]:
                combined_data[first_qa_type][response_type] = {
                    "factuality": {
                        "Yes": data[first_qa_type][response_type]["factuality"]["Yes"] + data[second_qa_type][response_type]["factuality"]["Yes"],
                        "No": data[first_qa_type][response_type]["factuality"]["No"] + data[second_qa_type][response_type]["factuality"]["No"]
                    },
                    "relevance": {
                        "Yes": data[first_qa_type][response_type]["relevance"]["Yes"] + data[second_qa_type][response_type]["relevance"]["Yes"],
                        "No": data[first_qa_type][response_type]["relevance"]["No"] + data[second_qa_type][response_type]["relevance"]["No"]
                    }
                }
            
            # Remove the original two qa_types after combining
            del data[first_qa_type]
            del data[second_qa_type]
    
    # Add combined data back to the main data dictionary
    data.update(combined_data)

    return data


def output_combined_data(data):
    # Output the data in the required format
    for qa_type, response_data in data.items():
        print(f"qa_type: {qa_type}")
        for response_type, counts in response_data.items():
            factuality_yes = counts["factuality"]["Yes"]
            factuality_no = counts["factuality"]["No"]
            relevance_yes = counts["relevance"]["Yes"]
            relevance_no = counts["relevance"]["No"]
            
            # Output the counts in the desired format
            print(f"  {response_type.replace('_', ' ').title()} - Factuality: {factuality_yes}/{factuality_no}")
            print(f"  {response_type.replace('_', ' ').title()} - Relevance: {relevance_yes}/{relevance_no}")
        print()  # Line break for better readability

    print(data)  # Print the combined data

# Original data
data = {
    'generate_qa_comparison_party': {'cosine_RAG_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 12, 'No': 4}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 13, 'No': 3}, 'relevance': {'Yes': 15, 'No': 1}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 10, 'No': 7}, 'relevance': {'Yes': 15, 'No': 2}}},
    'generate_qa_comparison_person': {'cosine_RAG_response': {'factuality': {'Yes': 17, 'No': 4}, 'relevance': {'Yes': 18, 'No': 3}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 16, 'No': 5}, 'relevance': {'Yes': 15, 'No': 6}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 13, 'No': 8}, 'relevance': {'Yes': 19, 'No': 2}}},
    'generate_qa_inference_person': {'cosine_RAG_response': {'factuality': {'Yes': 7, 'No': 7}, 'relevance': {'Yes': 8, 'No': 6}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 9, 'No': 5}, 'relevance': {'Yes': 9, 'No': 5}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 9, 'No': 5}, 'relevance': {'Yes': 8, 'No': 6}}},
    'generate_qa_inference_party': {'cosine_RAG_response': {'factuality': {'Yes': 3, 'No': 6}, 'relevance': {'Yes': 4, 'No': 5}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 4, 'No': 5}, 'relevance': {'Yes': 4, 'No': 5}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 3, 'No': 5}, 'relevance': {'Yes': 5, 'No': 3}}},
    'generate_qa_temporal': {'cosine_RAG_response': {'factuality': {'Yes': 1, 'No': 7}, 'relevance': {'Yes': 1, 'No': 7}}, 'graph_RAG_bm25_response': {'factuality': {'Yes': 5, 'No': 3}, 'relevance': {'Yes': 5, 'No': 3}}, 'graph_RAG_cosine_response': {'factuality': {'Yes': 4, 'No': 4}, 'relevance': {'Yes': 4, 'No': 4}}}
}

# Combine the qa_types as per the required pairs
combined_data = combine_qa_types(data)

# Output the combined data
output_combined_data(combined_data)


qa_type: generate_qa_temporal
  Cosine Rag Response - Factuality: 1/7
  Cosine Rag Response - Relevance: 1/7
  Graph Rag Bm25 Response - Factuality: 5/3
  Graph Rag Bm25 Response - Relevance: 5/3
  Graph Rag Cosine Response - Factuality: 4/4
  Graph Rag Cosine Response - Relevance: 4/4

qa_type: generate_qa_comparison_party
  Cosine Rag Response - Factuality: 30/7
  Cosine Rag Response - Relevance: 30/7
  Graph Rag Bm25 Response - Factuality: 29/8
  Graph Rag Bm25 Response - Relevance: 30/7
  Graph Rag Cosine Response - Factuality: 23/15
  Graph Rag Cosine Response - Relevance: 34/4

qa_type: generate_qa_inference_party
  Cosine Rag Response - Factuality: 10/13
  Cosine Rag Response - Relevance: 12/11
  Graph Rag Bm25 Response - Factuality: 13/10
  Graph Rag Bm25 Response - Relevance: 13/10
  Graph Rag Cosine Response - Factuality: 12/10
  Graph Rag Cosine Response - Relevance: 13/9

{'generate_qa_temporal': {'cosine_RAG_response': {'factuality': {'Yes': 1, 'No': 7}, 'relevance': {'Yes

In [ ]:
"/mnt/c/Users/User/thesis/detailed_output.csv"

In [9]:
import json
from collections import Counter


results = "/mnt/c/Users/User/thesis/data_import/data_small_size/data/qa_dataset_result.json"

with open(results, "r", encoding="utf-8") as f:
    data = json.load(f)

anforande_ids = [item.get("anforande_id") for item in data if "anforande_id" in item]
anforande_ids = set(anforande_ids)
print(anforande_ids)


qa_type_counts = Counter(item["qa_type"] for item in data if "qa_type" in item)
print(qa_type_counts)

{'d4bf82c6-d9fe-ec11-9174-901b0e9b71a0', '00839754-be2a-ed11-9178-901b0e9b71a8', 'a86f8c75-feb7-eb11-914f-901b0eac4c78', '2887ccf7-3728-ec11-915b-901b0eac4c78', '5048d815-7706-ec11-9160-901b0e9b71a0', '1b6f07f4-3a1d-ec11-915b-901b0eac4c78', '06c02348-bc23-ed11-9178-901b0e9b71a8', '04c790c4-7aba-ec11-9167-901b0eac4c78', 'fe87f68a-bb39-ed11-9178-901b0e9b71a8', 'b8a9e37b-c567-eb11-914e-901b0e9b71a0', '0efb26a5-23cd-eb11-9156-901b0e9b71a8', '0353ade4-36c2-ec11-9167-901b0eac4c78', 'db022d69-c139-ed11-9178-901b0e9b71a8', '4f137898-37ee-ec11-916f-901b0eac4c70', 'fffe7b0c-dafe-ec11-9174-901b0e9b71a0', '89ddcf4e-3dc6-ec11-9170-0090facf175a', '435e4e26-4157-eb11-9147-901b0eac4c70', '54449a9d-ad34-eb11-914a-901b0e9b71a8', 'fe550445-f0ca-ec11-9170-0090facf175a', 'a6184259-3c1d-ec11-915b-901b0eac4c78', '0f4078f4-bb23-ed11-9178-901b0e9b71a8', 'f64283c2-6fc8-eb11-9156-901b0e9b71a8', '62ea50f7-3328-ec11-915b-901b0eac4c78', 'be0c5c85-3a0b-ec11-9160-901b0e9b71a0', 'cd527919-c02d-ec11-915b-901b0eac4c78',